# Carbon Emission Prediction Model

In [90]:
from datetime import date
import torch # pip install torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import os
from ipynb.fs.full.train_benchmark import process_data; # pip install ipynb

## Dataset load and preprocess

In [91]:
# Set dataset locations
selected_output_variable = "OZONE"
data_file = "01_Data/02_Imagery/data_and_imagery_test.pkl"

In [92]:
X, y, res, num_channels, m = process_data(data_file, selected_output_variable) # from benchmark code
# reshape and convert datatype
X = torch.from_numpy(X.T).to(torch.float)
y = torch.from_numpy(y.T).to(torch.float)

# split dataset
percent_train = 0.80
num_train = int(percent_train * m)
num_test = m - num_train

# split dataset
X_train, X_test = torch.utils.data.random_split(X, [num_train, num_test])
y_train, y_test = torch.utils.data.random_split(y, [num_train, num_test])

# convert from Subset to Tensor type
X_train, X_test = X_train.dataset[X_train.indices], X_test.dataset[X_test.indices]
y_train, y_test = y_train.dataset[y_train.indices], y_test.dataset[y_test.indices]

## Define model

In [93]:
class Net(nn.Module):
    """
    Define the neural network: 1 hidden layer with ReLU activation
    """
    def __init__(self, res, num_channels, m):
        """
        Define the network layers
        """
        super(Net,self).__init__()
        self.layer1 = nn.Linear(res*res*num_channels, m) # computes W^T X + b
        
    def forward(self, x):
        """
        Define forward pass
        """
        x = torch.nn.functional.relu(self.layer1(x)) # ReLU activation
        return x

## Train model

In [111]:
def train(x_train, y_train, save=False, model_tag=''):
    """
    Train model on training data. If save=True, trained model is saved in ./03_Trained_Models/NN/model_date_<model_tag>.pt
    Input:
        x data and labels
    Output:
        trained model
    """
    print('Training...')
    optimizer.zero_grad() # clear gradients
    for epoch in range(10):
        output = net(x_train)
        loss = criterion(output, y_train)
        print('\tloss at epoch %i = %f' %(epoch, loss))
        loss.backward()
        optimizer.step()

    # save trained model
    if (save):
        d = date.today()
        path = os.path.join('03_Trained_Models', 'NN', 'model_%s_%s.pt' %(d, model_tag))
        torch.save(net.state_dict(), path)
    return net

## Main

In [116]:
# initialize network
net = Net(res, num_channels, m)
L = len(list(net.parameters())) # number of layers
# print(net)

# initialize loss function and optimizer
criterion = nn.MSELoss() # for regression
optimizer = optim.SGD(net.parameters(), lr=0.05)

# train model and save to ./03_Trained_Models/NN/model_date_<model_tag>.pt
model_tag = '1_hidden_relu'
train(X, y, True, model_tag)

# test model
print('Testing...')
output = net(X_test)
loss = criterion(output, y_test).item()
print('\tTest loss = ', loss)

Training...
	loss at epoch 0 = 760.013367
	loss at epoch 1 = 4379.063477
	loss at epoch 2 = 761.697632
	loss at epoch 3 = 761.697632
	loss at epoch 4 = 761.697632
	loss at epoch 5 = 761.697632
	loss at epoch 6 = 761.697632
	loss at epoch 7 = 761.697632
	loss at epoch 8 = 761.697632
	loss at epoch 9 = 761.697632
Testing...
	Test loss =  760.7791137695312
